<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Questions" data-toc-modified-id="Questions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Questions</a></span></li></ul></div>

## Questions

- How often have people published datasets in Root dataverse?
    - What are the account types of those users?
        - Among the non-shib user accounts, which have .edu in their email addresses?
    - How often have people created multiple datasets in Root? (User1 published a dataset in Root, then publishes one more additional datasets in Root. How often does this happen?)